In [1]:
import numpy as np
from PySDM.physics.constants import si
from settings import Settings
from simulation import Simulation
from PySDM import Formulae
import time

In [2]:
hom_nucleation_rate = "Koop_Correction"
formulae = Formulae(
            particle_shape_and_density="MixedPhaseSpheres",
            homogeneous_ice_nucleation_rate=hom_nucleation_rate,
            seed=time.time_ns()
        )

In [4]:
"""  basic setup for homogeneous freezing of pure-liquid droplets """
"""
Ensemble for:
1. n_sd
2. w
3. r & n_c
4. use diffrent hom. nucleation rates
Questions:
- where to store backend
- need to restart kernel if change in files outside notebook
"""
n_sd = 100
number_concentration_droplets = 1e4 / si.meter**3
mean_radius_droplets = 50 * si.micrometer


updraft = 1 * si.meter / si.second
initial_temperature =  formulae.trivia.C2K( -25. )
time_step = 0.1 * si.second
setting = Settings(
    formulae = formulae,
    n_sd = n_sd,
    w_updraft = updraft,
    T0 = initial_temperature,
    dt = time_step,
    N_dv_droplet_distribution = number_concentration_droplets,
    r_mean_droplet_distribution = mean_radius_droplets,
    type_droplet_distribution = "monodisperse",
    RH_0=1.,
    condensation_enable = False,
    deposition_enable = False,
)

simulation = Simulation(setting)


In [5]:
output = simulation.run()

9.99999999999998 248.05230144943604 1.8723617923038424e-05 0.0
20.000000000000014 247.95460289875845 1.872361795937764e-05 0.0
30.000000000000156 247.85690434796695 1.8723617995745507e-05 0.0
40.0000000000003 247.7592057970615 1.872361803214206e-05 0.0
50.00000000000044 247.66150724604208 1.8723618068567325e-05 0.0
60.00000000000058 247.5638086949085 1.872361810502134e-05 0.0
70.0000000000003 247.46611014366061 1.8723618141504136e-05 0.0
79.99999999999973 247.3684115922985 1.8723618178015752e-05 0.0
89.99999999999916 247.270713040822 1.4229949843062741e-05 4.4936683714935055e-06
99.9999999999986 247.17301448923106 0.0 1.872361825112558e-05
break due to LWC


In [14]:
print(output['IWC'])
print(output['rs'])
print(output['ri'])
print(output['T'])
print(output['T_frz'])

[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(4.4936683714935055e-06), np.float64(1.872361825112558e-05)]
[np.float64(49.99999999999994), np.float64(49.99999999999994), np.float64(49.99999999999994), np.float64(49.99999999999994), np.float64(49.99999999999994), np.float64(49.99999999999994), np.float64(49.99999999999994), np.float64(49.99999999999994), np.float64(49.99999999999994), np.float64(49.99999999999997), np.float64(0.0)]
[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(50.00000000000005), np.float64(49.99999999999994)]
[np.float64(248.14999999999992), np.float64(248.05230144943604), np.float64(247.95460289875845), np.float64(247.85690434796695), np.float64(247.7592057970615), np.float64(247.66150724604208), np.float64(247.5638086949085), np.float